# ETL Project using world happiness data from Kaggle.com

In [ ]:
# import dependencies
from pymongo import MongoClient
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sqlalchemy import create_engine

In [ ]:
from config import key

## Extract: import world happiness csv files

In [2]:
# create path to 2015 world happiness csv

file = "csv files/2015.csv"
df2015 = pd.read_csv(file)
df2015.head()

Country          Region  Happiness Rank  Happiness Score  Standard Error  Economy (GDP per Capita)   Family  Health (Life Expectancy)  Freedom  Trust (Government Corruption)  Generosity  Dystopia Residual
0  Switzerland  Western Europe               1            7.587         0.03411                   1.39651  1.34951                   0.94143  0.66557                        0.41978     0.29678            2.51738
1      Iceland  Western Europe               2            7.561         0.04884                   1.30232  1.40223                   0.94784  0.62877                        0.14145     0.43630            2.70201
2      Denmark  Western Europe               3            7.527         0.03328                   1.32548  1.36058                   0.87464  0.64938                        0.48357     0.34139            2.49204
3       Norway  Western Europe               4            7.522         0.03880                   1.45900  1.33095                   0.88521  0.66973                        0.36503     0.34699            2.46531
4       Canada   North America               5            7.427         0.03553                   1.32629  1.32261                   0.90563  0.63297                        0.32957     0.45811            2.45176

In [3]:
# create path to 2020 world happiness csv file

file = "csv files/2020.csv"
df2020 = pd.read_csv(file)
df2020.head()

Country name Regional indicator  Ladder score  Standard error of ladder score  upperwhisker  lowerwhisker  Logged GDP per capita  Social support  Healthy life expectancy  Freedom to make life choices  Generosity  Perceptions of corruption  Ladder score in Dystopia  Explained by: Log GDP per capita  Explained by: Social support  Explained by: Healthy life expectancy  Explained by: Freedom to make life choices  Explained by: Generosity  Explained by: Perceptions of corruption  Dystopia + residual
0      Finland     Western Europe        7.8087                        0.031156      7.869766      7.747634              10.639267        0.954330                71.900825                      0.949172   -0.059482                   0.195445                  1.972317                          1.285190                      1.499526                               0.961271                                    0.662317                  0.159670                                 0.477857             2.762835
1      Denmark     Western Europe        7.6456                        0.033492      7.711245      7.579955              10.774001        0.955991                72.402504                      0.951444    0.066202                   0.168489                  1.972317                          1.326949                      1.503449                               0.979333                                    0.665040                  0.242793                                 0.495260             2.432741
2  Switzerland     Western Europe        7.5599                        0.035014      7.628528      7.491272              10.979933        0.942847                74.102448                      0.921337    0.105911                   0.303728                  1.972317                          1.390774                      1.472403                               1.040533                                    0.628954                  0.269056                                 0.407946             2.350267
3      Iceland     Western Europe        7.5045                        0.059616      7.621347      7.387653              10.772559        0.974670                73.000000                      0.948892    0.246944                   0.711710                  1.972317                          1.326502                      1.547567                               1.000843                                    0.661981                  0.362330                                 0.144541             2.460688
4       Norway     Western Europe        7.4880                        0.034837      7.556281      7.419719              11.087804        0.952487                73.200783                      0.955750    0.134533                   0.263218                  1.972317                          1.424207                      1.495173                               1.008072                                    0.670201                  0.287985                                 0.434101             2.168266

## Transform: drop columns, rename columns, round decimals

In [4]:
# display 2015 columns

df2015.columns

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual'], dtype='object')

In [5]:
# round all numerical columns to two decimal points to clean up output 

def rounding(x):
    return round(x,2)

In [6]:
# drop unwanted columns from 2015 df

dropped2015 = df2015.drop(columns =['Region','Standard Error', 'Economy (GDP per Capita)', 'Family','Freedom', 'Dystopia Residual'])
dropped2015.head()

Country  Happiness Rank  Happiness Score  Health (Life Expectancy)  Trust (Government Corruption)  Generosity
0  Switzerland               1            7.587                   0.94143                        0.41978     0.29678
1      Iceland               2            7.561                   0.94784                        0.14145     0.43630
2      Denmark               3            7.527                   0.87464                        0.48357     0.34139
3       Norway               4            7.522                   0.88521                        0.36503     0.34699
4       Canada               5            7.427                   0.90563                        0.32957     0.45811

In [7]:
# display 2020 df columns

df2020.columns

Index(['Country name', 'Regional indicator', 'Ladder score', 'Standard error of ladder score', 'upperwhisker', 'lowerwhisker', 'Logged GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Ladder score in Dystopia', 'Explained by: Log GDP per capita', 'Explained by: Social support', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption', 'Dystopia + residual'], dtype='object')

In [8]:
# drop unwanted columns from 2020 df

dropped2020 = df2020.drop(columns = ['Regional indicator', 
       'Standard error of ladder score', 'upperwhisker', 'lowerwhisker',
       'Logged GDP per capita', 'Social support', 
       'Freedom to make life choices',
       'Ladder score in Dystopia',
       'Explained by: Log GDP per capita', 'Explained by: Social support',
       'Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption',
       'Dystopia + residual'])
dropped2020.head()

Country name  Ladder score  Healthy life expectancy  Generosity  Perceptions of corruption
0      Finland        7.8087                71.900825   -0.059482                   0.195445
1      Denmark        7.6456                72.402504    0.066202                   0.168489
2  Switzerland        7.5599                74.102448    0.105911                   0.303728
3      Iceland        7.5045                73.000000    0.246944                   0.711710
4       Norway        7.4880                73.200783    0.134533                   0.263218

In [9]:
# rename columns in 2015 dropped df

dropped2015_transformed = dropped2015.rename(columns={"Country": "country", 
                                                      "Happiness Rank": "happiness_rank_2015", 
                                                      "Happiness Score": "happiness_score_2015",
                                                          "Health (Life Expectancy)": "life_expectancy_2015",
                                                          "Generosity": "generosity_2015", "Trust (Government Corruption)": "trust_2015"})


dropped2015_ordered = dropped2015_transformed[["country", "happiness_rank_2015", "happiness_score_2015", "life_expectancy_2015", "generosity_2015", "trust_2015"]]
dropped2015_ordered.head()

country  happiness_rank_2015  happiness_score_2015  life_expectancy_2015  generosity_2015  trust_2015
0  Switzerland                    1                 7.587               0.94143          0.29678     0.41978
1      Iceland                    2                 7.561               0.94784          0.43630     0.14145
2      Denmark                    3                 7.527               0.87464          0.34139     0.48357
3       Norway                    4                 7.522               0.88521          0.34699     0.36503
4       Canada                    5                 7.427               0.90563          0.45811     0.32957

In [10]:
# round numerical columns to two decimal points

dropped2015_round = dropped2015_ordered
dropped2015_round[["happiness_score_2015", "life_expectancy_2015", "generosity_2015", "trust_2015"]] = dropped2015_round[["happiness_score_2015", "life_expectancy_2015", "generosity_2015", "trust_2015"]].apply(rounding)
dropped2015_round.head()

country  happiness_rank_2015  happiness_score_2015  life_expectancy_2015  generosity_2015  trust_2015
0  Switzerland                    1                  7.59                  0.94             0.30        0.42
1      Iceland                    2                  7.56                  0.95             0.44        0.14
2      Denmark                    3                  7.53                  0.87             0.34        0.48
3       Norway                    4                  7.52                  0.89             0.35        0.37
4       Canada                    5                  7.43                  0.91             0.46        0.33

In [11]:
# rename columns in 2020 dropped df

dropped2020_transformed = dropped2020.rename(columns={"Country name": "country", 
                                                      "Ladder score": "happiness_score_2020",
                                                        "Healthy life expectancy": "life_expectancy_2020",
                                                          "Generosity": "generosity_2020",
                                                          "Perceptions of corruption": "trust_2020"})
dropped2020_transformed.head()

country  happiness_score_2020  life_expectancy_2020  generosity_2020  trust_2020
0      Finland                7.8087             71.900825        -0.059482    0.195445
1      Denmark                7.6456             72.402504         0.066202    0.168489
2  Switzerland                7.5599             74.102448         0.105911    0.303728
3      Iceland                7.5045             73.000000         0.246944    0.711710
4       Norway                7.4880             73.200783         0.134533    0.263218

In [12]:
# round numerical columns to two decimal points

dropped2020_round = dropped2020_transformed
dropped2020_round[["happiness_score_2020", "life_expectancy_2020", "generosity_2020", "trust_2020"]] = dropped2020_round[["happiness_score_2020", "life_expectancy_2020", "generosity_2020", "trust_2020"]].apply(rounding)
dropped2020_round.head()

country  happiness_score_2020  life_expectancy_2020  generosity_2020  trust_2020
0      Finland                  7.81                  71.9            -0.06        0.20
1      Denmark                  7.65                  72.4             0.07        0.17
2  Switzerland                  7.56                  74.1             0.11        0.30
3      Iceland                  7.50                  73.0             0.25        0.71
4       Norway                  7.49                  73.2             0.13        0.26

## Transform: merge 2015 and 2020 dataframes to check for null values 

In [13]:
# merge dataframes to check for null values

merged_df = pd.merge(dropped2015_ordered, dropped2020_transformed, on="country", how="outer")
merged_df.head(158)

country  happiness_rank_2015  happiness_score_2015  life_expectancy_2015  generosity_2015  trust_2015  happiness_score_2020  life_expectancy_2020  generosity_2020  trust_2020
0                 Switzerland                  1.0                  7.59                  0.94             0.30        0.42                  7.56                 74.10             0.11        0.30
1                     Iceland                  2.0                  7.56                  0.95             0.44        0.14                  7.50                 73.00             0.25        0.71
2                     Denmark                  3.0                  7.53                  0.87             0.34        0.48                  7.65                 72.40             0.07        0.17
3                      Norway                  4.0                  7.52                  0.89             0.35        0.37                  7.49                 73.20             0.13        0.26
4                      Canada                  5.0                  7.43                  0.91             0.46        0.33                  7.23                 73.60             0.12        0.39
5                     Finland                  6.0                  7.41                  0.89             0.23        0.41                  7.81                 71.90            -0.06        0.20
6                 Netherlands                  7.0                  7.38                  0.89             0.48        0.32                  7.45                 72.30             0.21        0.36
7                      Sweden                  8.0                  7.36                  0.91             0.36        0.44                  7.35                 72.60             0.11        0.25
8                 New Zealand                  9.0                  7.29                  0.91             0.48        0.43                  7.30                 73.20             0.19        0.22
9                   Australia                 10.0                  7.28                  0.93             0.44        0.36                  7.22                 73.60             0.19        0.42
10                     Israel                 11.0                  7.28                  0.91             0.33        0.08                  7.13                 73.20             0.10        0.78
11                 Costa Rica                 12.0                  7.23                  0.86             0.25        0.11                  7.12                 71.30            -0.10        0.79
12                    Austria                 13.0                  7.20                  0.89             0.33        0.19                  7.29                 73.00             0.09        0.50
13                     Mexico                 14.0                  7.19                  0.81             0.14        0.21                  6.47                 68.30            -0.18        0.81
14              United States                 15.0                  7.12                  0.86             0.40        0.16                  6.94                 68.30             0.15        0.70
15                     Brazil                 16.0                  6.98                  0.70             0.15        0.18                  6.38                 66.48            -0.10        0.77
16                 Luxembourg                 17.0                  6.95                  0.92             0.28        0.38                  7.24                 72.60            -0.00        0.37
17                    Ireland                 18.0                  6.94                  0.90             0.46        0.29                  7.09                 72.30             0.15        0.36
18                    Belgium                 19.0                  6.94                  0.90             0.22        0.23                  6.86                 72.00            -0.08        0.61
19       United Arab Emirates                 20.0                  6.90          

## Load: connect to PostgreSQL Database to Load the Dataframe

In [ ]:
# connect to postgres sql server and database via sql alchemy

engine = create_engine(key, echo=False)
connection = engine.connect()

In [ ]:
# get table names

engine.table_names()

In [ ]:
# load 2015 dataframe into postgresql database

dropped2015_ordered.to_sql(name='happiness_2015', con=engine, if_exists='append', index=False)

In [ ]:
# load 2020 dataframe into postgresql database

dropped2020_transformed.to_sql(name='happiness_2020', con=engine, if_exists='append', index=False)

In [ ]:
# confirm data has been loaded into 2015 happiness table

pd.read_sql_query('select * from happiness_2015', con=engine).head()

In [ ]:
# confirm data has been loaded into 2020 happiness table

pd.read_sql_query('select * from happiness_2020', con=engine).head()

## Load: import dataframes into mongoDB database using pymongo

In [14]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)

# Insert 2015 data into mongoDB

try: 
    conn = client
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

data = dropped2015_ordered.to_dict(orient='records') 

try: 
    db = client['world_happiness_db']
    collection = db['2015_world_happiness']
    # Insert collection
    collection.insert_many(data)
    print("2015 Data inserted successfully!!")
except: 
    print("2015 Data upload failed")


Connected successfully!!!
2015 Data inserted successfully!!


In [15]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)

# Insert 2020 data into mongoDB

try: 
    conn = client
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

data2020 = dropped2020_transformed.to_dict(orient='records') 

try: 
    db = client['world_happiness_db']
    collection = db['2020_world_happiness']
    # Insert collection
    collection.insert_many(data2020)
    print("2020 Data inserted successfully!!")
except: 
    print("Data upload failed")

Connected successfully!!!
2020 Data inserted successfully!!
